# LangChain 컴포넌트

In [1]:
%pip install -qU langchain langchain-openai langchain-community faiss-cpu google-search-results


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## 모델과 프롬프트

In [3]:
from langchain_openai import OpenAI

llm = OpenAI()


In [5]:
llm.invoke('AI Agent 설명')

'\n\nAI Agent는 사용자의 대화를 이해하고 그에 따른 행동을 수행할 수 있는 소프트웨어 에이전트이다. AI 기술을 이용하여 사용자의 언어를 이해하고, 그에 따라 적절한 응답을 제공하거나 원하는 작업을 수행할 수 있다.\n\nAI Agent는 다양한 형태로 존재할 수 있으며, 가장 대표적으로는 챗봇(chatbot)이 있다. 챗봇은 사용자와 실시간으로 대화하면서 사용자의 요청에 따라 정보를 제공하거나 서비스를 제공할 수 있는 AI Agent이다.\n\n또 다른 형태로는 가상 보조 기기(voice assistant)가 있다. 가상 보조 기기는 음성 인식 기술을 이용하여 사용자의 음성 명령을 인식하고, 그에 따라 사용'

In [7]:
from langchain import PromptTemplate

template = " '{sentence}' 를 {language}로 번역하세요."
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "화창하고 여유로운 금요일에", language = "영어"))

 '화창하고 여유로운 금요일에' 를 영어로 번역하세요.


## Data Connector

### Document Loader

In [8]:
import csv

# 샘플 데이터
data = [
    ['이름', '나이', '도시'],
    ['김민기', 25, '서울'],
    ['이지성', 23, '대전'],
    ['강동윤', 32, '대구']
]


# 데이터를 CSV 파일에 저장
file_name = 'data.csv'
with open(file_name, 'w', newline='') as file:
    csv_file = csv.writer(file)
    csv_file.writerows(data)

print(f'데이터를 "{file_name}" 파일로 저장하였습니다.')



데이터를 "data.csv" 파일로 저장하였습니다.


In [9]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='data.csv')
data = loader.load()

print(data)

[Document(metadata={'source': 'data.csv', 'row': 0}, page_content='이름: 김민기\n나이: 25\n도시: 서울'), Document(metadata={'source': 'data.csv', 'row': 1}, page_content='이름: 이지성\n나이: 23\n도시: 대전'), Document(metadata={'source': 'data.csv', 'row': 2}, page_content='이름: 강동윤\n나이: 32\n도시: 대구')]


### Text Splitter

In [10]:
# 샘플 문장
sentence = """LangChain(랭체인)은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발에 특화된 오픈소스 프레임워크입니다.
LangChain은 기존 언어 모델의 한계를 극복하고, AI 기술을 활용한 새로운 애플리케이션을 구축할 수 있는 중요한 도구로 자리 잡고 있습니다.
기존의 언어 모델이 주로 텍스트 생성에 중점을 둔 반면, LangChain은 다양한 외부 데이터 소스와 통합하여 보다 복잡하고 유용한 애플리케이션을 만들 수 있도록 설계되었습니다.
LLM의 잠재력을 극대화하기 위해 데이터베이스, 파일 시스템 등과 같은 다양한 데이터 소스와의 통합을 지원하여, 실시간 데이터와 상호작용하는 애플리케이션을 구축할 수 있습니다.
이러한 통합은 언어 모델이 단순한 텍스트 응답을 넘어, 사용자에게 보다 정확하고 유의미한 정보를 제공하는 애플리케이션을 가능하게 합니다.
LangChain을 통해 개발자는 더 복잡한 사용 사례를 처리할 수 있는 애플리케이션을 손쉽게 구축할 수 있습니다."""

# 샘플 문장을 텍스트 파일에 저장
file_name = 'langchain.txt'
with open(file_name, 'w') as file:
    file.write(sentence)

with open('langchain.txt') as f:
    langchain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len
)

docs = text_splitter.create_documents([langchain])

print(docs[0])
print(docs[1])


page_content='LangChain(랭체인)은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발에 특화된 오픈소스 프레임워크입니다.'
page_content='LangChain은 기존 언어 모델의 한계를 극복하고, AI 기술을 활용한 새로운 애플리케이션을 구축할 수 있는 중요한 도구로 자리 잡고 있습니다.'


### 텍스트 임베딩 모델

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model ='text-embedding-3-small' )
embeddings = embeddings_model.embed_documents(
    [
        "LangChain은 LLM을 효과적으로 활용하기 위한 유연한 프롬프팅과 컨텍스트 관리 도구를 제공합니다. ",
        "이러한 도구는 언어 모델이 사용자 질문에 대해 더욱 정확하고 맞춤형 응답을 생성할 수 있도록 돕습니다. ",
        "프롬프팅 기능은 사용자와의 대화를 보다 자연스럽고 연속성 있게 만들어주며, 복잡한 대화 흐름에서도 일관된 컨텍스트를 유지할 수 있게 합니다. ",
        "이를 통해 고객 지원 챗봇이나 인터랙티브한 학습 도우미와 같은 애플리케이션에서 사용자 경험을 크게 향상할 수 있습니다."
    ]
)

In [12]:
print(f"임베딩 벡터 갯수 : {len(embeddings)}; 임베딩 벡터 차원 : {len(embeddings[0])}")


임베딩 벡터 갯수 : 4; 임베딩 벡터 차원 : 1536


In [13]:
embedded_query = embeddings_model.embed_query("LangChain 기능은?")

print("임베드 질의:")
print(f"임베딩 벡터 차원 : {len(embedded_query)}")
print(f"임베딩 벡터 : {embedded_query}")

임베드 질의:
임베딩 벡터 차원 : 1536
임베딩 벡터 : [0.01351103000342846, 0.03553810343146324, 0.010241477750241756, 0.0026714990381151438, 0.025711899623274803, 0.016037767753005028, -0.0489906445145607, -0.0010827833320945501, -0.025688502937555313, -0.011768048629164696, 0.006088736932724714, 0.01615474745631218, 0.002903994172811508, 0.02182820998132229, -0.022764038294553757, -0.042018719017505646, -0.0519385039806366, 0.02353609725832939, 0.0033836064394563437, 0.04435829073190689, -0.009820355102419853, -0.0038719922304153442, 0.024308156222105026, 0.038298796862363815, -0.007059659343212843, -0.0224715918302536, 0.011914271861314774, 0.026881685480475426, -0.02653075009584427, -0.04520053789019585, 0.014294786378741264, -0.042018719017505646, 0.07678475975990295, -0.008229445666074753, -0.004187834449112415, 0.04367981478571892, 0.03549131006002426, 0.012306150048971176, 0.009791109710931778, 0.027747327461838722, -0.0008919619140215218, 0.040965911000967026, -0.00889622326940298, -0.02421457320

In [43]:
# 대화 내용
dialogue = [
    "고객: 안녕하세요. 며칠 전에 귀사에서 제품을 구매했는데, 사용해 보니 불량이 있어서 환불을 요청하려고 합니다. ",
    "상담원: 안녕하세요, 고객님. 이용에 불편을 드려 죄송합니다. 어떤 제품을 구매하셨고, 정확히 어떤 문제가 발생했는지 말씀해 주시겠어요? ",
    "고객: 제가 지난주에 온라인몰에서 무선 이어폰을 구매했는데, 한쪽 이어폰에서 소리가 전혀 나오지 않아요. ",
    "상담원: 확인해 주셔서 감사합니다. 고객님의 불편을 해결해 드리기 위해 몇 가지 확인이 필요합니다. 혹시 충전 상태를 확인해 보셨나요? ",
    "그리고 다른 기기에서도 동일한 문제가 발생하나요? ",
    "고객: 네, 충전도 충분히 했고, 다른 스마트폰에도 연결해 봤지만 한쪽에서 소리가 나오지 않더라고요. ",
    "상담원: 알겠습니다. 고객님께서 받은 제품이 초기 불량일 가능성이 있습니다. 환불을 원하시면, 제품을 반품해 주시면 확인 후 환불 절차를 진행해 드리겠습니다. ",
    "고객: 네, 환불을 원합니다. 반품은 어떻게 하면 되나요? ",
    "상담원: 고객님의 주문 정보를 확인한 후 반품 접수를 도와드리겠습니다. 주문번호를 알려주실 수 있나요? ",
    "고객: 네, 주문번호는 20250405-12345입니다. ",
    "상담원: 확인되었습니다. 고객님께서 제품을 원래 포장 그대로 유지한 상태로 동봉된 반품 안내서에 따라 반송해 주시면 됩니다. 반품 배송비는 제품 불량으로 인한 환불이므로 판매자가 부담합니다. 반품이 확인되면 3~5영업일 이내에 환불 처리가 완료될 예정입니다. ",
    "고객: 알겠습니다. 반품 보내면 별도로 연락해야 하나요? ",
    "상담원: 아니요, 고객님께서 반품을 보내주시면 자동으로 접수되며, 처리 과정은 문자 또는 이메일로 안내해 드리겠습니다. ",
    "고객: 네, 감사합니다. ",
    "상담원: 이용에 불편을 드려 다시 한번 죄송합니다. 더 궁금한 점 있으시면 언제든지 문의 주세요. 좋은 하루 보내세요! "
]

# 대화 목록을 파일에 저장
file_name = 'dialogue.txt'
with open(file_name, 'w') as file:
    for line in dialogue:
        file.write(line + '\n')

print(f'대화 내용을 "{file_name}" 에 저장하였습니다.')


대화 내용을 "dialogue.txt" 에 저장하였습니다.


### 벡터 스토어

In [44]:
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

# 문서를 청크로 분할하고, 각 청크를 임베딩벡터로 변환하여 벡터스토어에 저장
raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

Created a chunk of size 63, which is longer than the specified 50
Created a chunk of size 77, which is longer than the specified 50
Created a chunk of size 59, which is longer than the specified 50
Created a chunk of size 76, which is longer than the specified 50
Created a chunk of size 57, which is longer than the specified 50
Created a chunk of size 89, which is longer than the specified 50
Created a chunk of size 58, which is longer than the specified 50
Created a chunk of size 145, which is longer than the specified 50
Created a chunk of size 67, which is longer than the specified 50


In [45]:
query = "반품비용을 누가 부담하는가?"
docs = db.similarity_search(query)

In [46]:
print(docs)

[Document(id='f457a8a8-d47b-40b6-8f0e-3b4dd97ac254', metadata={'source': 'dialogue.txt'}, page_content='고객: 네, 환불을 원합니다. 반품은 어떻게 하면 되나요?'), Document(id='c5957a87-c529-4a3b-b9c1-3fc55a50d8b9', metadata={'source': 'dialogue.txt'}, page_content='고객: 알겠습니다. 반품 보내면 별도로 연락해야 하나요?'), Document(id='16ff6579-f45a-4212-8e08-24a2867e29f1', metadata={'source': 'dialogue.txt'}, page_content='상담원: 확인되었습니다. 고객님께서 제품을 원래 포장 그대로 유지한 상태로 동봉된 반품 안내서에 따라 반송해 주시면 됩니다. 반품 배송비는 제품 불량으로 인한 환불이므로 판매자가 부담합니다. 반품이 확인되면 3~5영업일 이내에 환불 처리가 완료될 예정입니다.'), Document(id='eb33b2cd-c520-43e5-97f1-13fd9c161cfa', metadata={'source': 'dialogue.txt'}, page_content='상담원: 고객님의 주문 정보를 확인한 후 반품 접수를 도와드리겠습니다. 주문번호를 알려주실 수 있나요?')]


In [47]:
print(docs[0])

page_content='고객: 네, 환불을 원합니다. 반품은 어떻게 하면 되나요?' metadata={'source': 'dialogue.txt'}


In [48]:
print(docs[2].page_content)

상담원: 확인되었습니다. 고객님께서 제품을 원래 포장 그대로 유지한 상태로 동봉된 반품 안내서에 따라 반송해 주시면 됩니다. 반품 배송비는 제품 불량으로 인한 환불이므로 판매자가 부담합니다. 반품이 확인되면 3~5영업일 이내에 환불 처리가 완료될 예정입니다.


### Retrieval

In [49]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

retriever = db.as_retriever()

In [50]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "반품비용을 누가 부담하는가?"
output = qa.invoke(query)
output['result']

' 판매자가 부담합니다.'

## Memory

In [52]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "안녕하세요, AI에이전트 활용 아이디어를 찾고 있어요"}, {"output": "고객센터 서비스 에 활용해보세요."})

memory.load_memory_variables({})

{'history': '\nThe human is looking for ideas to utilize AI agents. The AI suggests using them for customer service.'}

### LangGraph 메모리로 마이그레이션하는 방법 참고
* https://python.langchain.com/docs/versions/migrating_memory/

## LLMChain (기본 체인)
- 프롬프트를 받아서 LLM에 전달한 후, 결과를 출력하는 가장 기본적인 체인

In [54]:
from langchain_openai import OpenAI
from langchain import PromptTemplate, LLMChain

template = " '{sentence}' 를 {language}로 번역하세요."
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])
llm = OpenAI()
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.predict(sentence="화창하고 화사한 금요일입니다.", language="영어")

"\n\nIt's a bright and cheerful Friday."

In [56]:
# LLMChain deprecation 해결

from langchain_openai import OpenAI
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

template = " '{sentence}' 를 {language}로 번역하세요."
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])
llm = OpenAI(temperature=0)
output_parser = StrOutputParser()

# chain = prompt | llm | output_parser
chain = RunnableSequence(
    {
        "sentence": RunnablePassthrough(),
        "language": RunnablePassthrough()
    }
    | prompt
    | llm
    | output_parser
)

result = chain.invoke({
    "sentence": "화사하고 화창한 금요일입니다.",
    "language": "영어"
})

print(result)



{'sentence': 'It is a bright and sunny Friday.', 'language': 'English'}


### RouterChain (라우팅 체인)
- 입력에 따라 적절한 체인을 선택하여 실행
- 여러 개의 체인 중에서 입력 조건에 따라 가장 적절한 체인을 실행하는 방식

In [57]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = OpenAI()

itinerary_template = """당신은 여행 플래너 입니다. \
고객의 최고의 여행을 할 수 있는 여행지를 찾고 여행 \일정 플래닝을 도와줍니다. \
당신은 고객의 선호에 따라 최적화된 여행 일정을 계획하는 데 도움을 줍니다.

요청사항:
{input}"""

restaurant_template = """당신은 레스토랑 예약 도우미입니다. \
당신은 고객의 수와 고객이 선호하는 음식을 확인합니다. \
그리고, 특별한 조건을 고려해야 하는지 주의합니다.

요청사항:
{input}"""

prompt_infos = [
    {
        "name": "여행 일정",
        "description": "여행 일정 플래닝을 돕습니다",
        "prompt_template": itinerary_template,
    },
    {
        "name": "레스토랑",
        "description": "고객의 레스토랑 예약을 도와줍니다",
        "prompt_template": restaurant_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

/var/folders/jy/_npvs4nx55d4n_gzbg5q40500000gn/T/ipykernel_2949/1088819059.py:46: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm=llm, output_key="text")
/Users/danny/cdt/venv/lib/python3.12/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/var/folders/jy/_npvs4nx55d4n_gzbg5q40500000gn/T/ipykernel_2949/1088819059.py:58: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = Mul

In [62]:
print(chain.invoke("이탈리아 밀라노에서 베니스까지 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?."))



> Entering new MultiPromptChain chain...
여행 일정: {'input': '이탈리아 밀라노에서 베니스까지 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?.'}
> Finished chain.
{'input': '이탈리아 밀라노에서 베니스까지 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?.', 'text': '\n\n당신은 이탈리아 여행의 전문가이기 때문에 이탈리아 여행지 중에서도 가장 인기있는 명소들을 알고 있을 것입니다. 여행 일정에 베니스를 포함하고자 한다면 중간에 들르기 좋은 곳으로는 베로나, 피사, 피렌체 등이 있습니다. 이들 곳은 유서 깊은 역사와 아름다운 건축물, 그리고 풍부한 음식 문화를 갖춘 도시로 많은 여행객들이 찾는 인기 있는 여행지입니다. 베로나는 로미오와 줄리엣의 사랑 이야기로 유명한 도시로서 로맨틱한 분위기를 느낄 수 있으며, 피사는 세계에서 가장 유명한 사이트 중 하나'}


In [63]:
print(chain.invoke("오늘 저녁 식사를 예약하고 싶어요"))



> Entering new MultiPromptChain chain...
레스토랑: {'input': '오늘 저녁 식사를 예약하고 싶어요'}
> Finished chain.
{'input': '오늘 저녁 식사를 예약하고 싶어요', 'text': '. 총 4명이고, 한식을 선호합니다.\n\n고객님, 오늘 저녁 식사 예약 감사합니다. 총 4명이신데, 한식을 선호하신다고 하셨네요. 저희 레스토랑에는 다양한 한식 메뉴가 준비되어 있으니 맘에 드는 메뉴를 선택해주시면 될 것 같습니다. 또한, 고객님들의 특별한 요청사항이 있으시다면 언제든지 말씀해주세요. 저희 레스토랑에서 최선을 다해 도와드리겠습니다. 즐거운 식사 되시길 바랍니다.'}


### SequentialChain (연속 체인)
- 여러 개의 체인을 순차적으로 실행   
- 하나의 체인의 출력을 다음 체인의 입력으로 전달하는 방식  
- 이전 체인의 결과를 활용하여 점진적으로 정보를 확장하는 데 유용

In [64]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=0)

template = """당신은 코미디언입니다. {topic}에 관한 농담을 생성하세요.
농담:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.
번역:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [65]:
# 체인을 순서대로 실행하는 전체 체인
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("개와 고양이")

/var/folders/jy/_npvs4nx55d4n_gzbg5q40500000gn/T/ipykernel_2949/4027272763.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  translated_joke = overall_chain.run("개와 고양이")




> Entering new SimpleSequentialChain chain...
 "개와 고양이가 싸우면 누가 이길까요? 당연히 주인이겠죠! 왜냐하면 그들은 주인의 사랑을 두고 싸우기 때문이죠."
 If dogs and cats fight, who do you think will win? Of course, the owner will win! Because they are fighting for the love of their owner.

> Finished chain.


### TransformChain (데이터 변환 체인)
- LLM을 사용하지 않고 데이터 변환을 수행하는 체인
- 텍스트를 전처리하거나, 데이터를 가공하는 용도로 사용

In [66]:
import string

def rename_cat(inputs: dict) -> dict:
    text = inputs["text"]
    new_text = text.replace('고객', '손님')
    return {"output_text": new_text}

In [67]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

with open("dialogue.txt") as f:
    cats_and_dogs = f.read()

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=rename_cat
)

template = """이 텍스트를 요약하세요:

{output_text}

요약:"""

prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])
sequential_chain.invoke(cats_and_dogs)

{'input': '고객: 안녕하세요. 며칠 전에 귀사에서 제품을 구매했는데, 사용해 보니 불량이 있어서 환불을 요청하려고 합니다. \n상담원: 안녕하세요, 고객님. 이용에 불편을 드려 죄송합니다. 어떤 제품을 구매하셨고, 정확히 어떤 문제가 발생했는지 말씀해 주시겠어요? \n고객: 제가 지난주에 온라인몰에서 무선 이어폰을 구매했는데, 한쪽 이어폰에서 소리가 전혀 나오지 않아요. \n상담원: 확인해 주셔서 감사합니다. 고객님의 불편을 해결해 드리기 위해 몇 가지 확인이 필요합니다. 혹시 충전 상태를 확인해 보셨나요? \n그리고 다른 기기에서도 동일한 문제가 발생하나요? \n고객: 네, 충전도 충분히 했고, 다른 스마트폰에도 연결해 봤지만 한쪽에서 소리가 나오지 않더라고요. \n상담원: 알겠습니다. 고객님께서 받은 제품이 초기 불량일 가능성이 있습니다. 환불을 원하시면, 제품을 반품해 주시면 확인 후 환불 절차를 진행해 드리겠습니다. \n고객: 네, 환불을 원합니다. 반품은 어떻게 하면 되나요? \n상담원: 고객님의 주문 정보를 확인한 후 반품 접수를 도와드리겠습니다. 주문번호를 알려주실 수 있나요? \n고객: 네, 주문번호는 20250405-12345입니다. \n상담원: 확인되었습니다. 고객님께서 제품을 원래 포장 그대로 유지한 상태로 동봉된 반품 안내서에 따라 반송해 주시면 됩니다. 반품 배송비는 제품 불량으로 인한 환불이므로 판매자가 부담합니다. 반품이 확인되면 3~5영업일 이내에 환불 처리가 완료될 예정입니다. \n고객: 알겠습니다. 반품 보내면 별도로 연락해야 하나요? \n상담원: 아니요, 고객님께서 반품을 보내주시면 자동으로 접수되며, 처리 과정은 문자 또는 이메일로 안내해 드리겠습니다. \n고객: 네, 감사합니다. \n상담원: 이용에 불편을 드려 다시 한번 죄송합니다. 더 궁금한 점 있으시면 언제든지 문의 주세요. 좋은 하루 보내세요! \n',
 'output': '\n손님은 며칠 전 불량이 발생한 제품으로 환불을 요청한다. 상담원은 확인을 위해 제

## Agent
- Agent(에이전트)는 LLM이 자체적으로 의사 결정을 하면서 여러 도구를 동적으로 활용할 수 있도록 해주는 기능
- 단순히 프롬프트에 대한 응답 생성이 아니라, 외부 API, 데이터베이스, 계산기, 웹 검색 등의 도구를 선택하여 실행

SERPAPI_API_KEY : https://serper.dev/api-key

In [80]:
import os
import getpass

if "SERPAPI_API_KEY" not in os.environ:
    os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your Serp API key: ")


In [81]:
from langchain_community.utilities.serpapi import SerpAPIWrapper

search = SerpAPIWrapper()

In [83]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain_openai import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

tools = [Tool.from_function(
            func=search.run,
            name="Search",
            description="이벤트에 관해 질문할 때 유용함"
        )
    ]

agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.invoke("오징어2 개봉일은?")



> Entering new AgentExecutor chain...
 어떤 정보를 알고 싶은지 생각해본 후에 어떤 도구를 사용해야 하는지 생각해본다.
Action: Search
Action Input: 오징어2 개봉일
Observation: 개봉 오징어 게임 시즌 2는 2024년 12월 26일에 개봉되었다.
Thought: 게임 시즌 2의 개봉일을 알아냈다.
Final Answer: 오징어2 개봉일은 2024년 12월 26일이다.

> Finished chain.


{'input': '오징어2 개봉일은?', 'output': '오징어2 개봉일은 2024년 12월 26일이다.'}